In [3]:
%matplotlib inline

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import geemap
import ee


In [4]:
ee.Authenticate()


Successfully saved authorization token.


In [5]:
ee.Initialize()

In [11]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [32]:
import folium

In [34]:
# Use Sentinel-2 data.
BANDS = ['NDVI', 'WDRVI', 'SAVI']
def add_indexes(img): 
    ndvi = img.expression(
        '(nir - red) / (nir  + red + a)', {
            'a': 1e-5,
            'nir': img.select('B8'),
            'red': img.select('B4')
        }
       
    ).rename('NDVI')

    wdrvi = img.expression(
        '(a * nir - red) / (a * nir + red)', {
            'a': 0.2,
            'nir': img.select('B8'),
            'red': img.select('B4')
        }
    ).rename('WDRVI')

    savi = img.expression(
        '1.5 * (nir - red) / (nir + red + 0.5)', {
            'nir': img.select('B8'),
            'red': img.select('B4')
        }
    ).rename('SAVI')

    return ee.Image.cat([ndvi, wdrvi, savi])

image = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2018-01-01', '2018-04-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(add_indexes) \
    .median()

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': BANDS, 'min': -1, 'max': 1})
map = folium.Map(location=[              
              9.835828,
              0.162381,
])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

map.add_child(folium.LayerControl())
map


In [26]:
 # Add necessary libraries.
!pip install -q focal-loss
import os
from os import path as op
import tensorflow as tf
import folium
from focal_loss import SparseCategoricalFocalLoss


IndexError: list index out of range